# Main Data Preparation

This notebook is dedicated to the initial/main data preparation for the prediction task at hand. Some important steps completed are as follows:


* __Aggregate data to a monthly level__
    * Explanation: As illustrated in the xgboost_daily notebook, I completed an initial forecast on a daily level. After investigating the results it became clear that because of the zero inflated nature of the daily training data, any prediction algorith would struggle to do better than just predicting the majortiy class (no sales). As a result, and still inline with the requested assignment, I decided to aggregate sales to a monthly level in order to create a more balanced dataset.
</br>
</br>
* __Limit the data to products sold in 2020__
    * Explanation: In order to cleanup the data and reduce the amount of data we need to work with in order to satisfy memory constraints, I limit the data to only include products that were sold in 2020. The logic behind this reduction is that products that have not been sold in the first 5 months of 2020 do not require forecasting. Their forecast is simply 0.
</br>
</br>
* __Limit the dataset to the past 15 months__
    * Explanation: The motivation for excluding data prior to 2019-03-01 was twofold. 
        * First, it is a question of relevance. How relevant is the sales of a product 2 years prior to the sales of the product today (especially compared to the relevance of more recent sales)? As such, the first motivation to exclude this data was to aid the ml model to focus on more recent data points. 
        * The second motivation was again one of memory limitation (to reduce data size). 
</br>
</br>
* __Add entries for days on which a product was not sold__
    * Explanation: The received dataset contained entries for each product only on days for which it was sold. The zero sale days for each product-shop combination is missing, and very relevant information for properly training a forecasting model. In constructing these mising data points, the following assumption was addopted:
        * On a day that a product was not sold it is assumed that the price was as on its previous sell date. If this is not available, it is set equal to its next sell price instance
</br>
</br>
* __Separate data by shop__
    * Explanation: The received dataset contained information related to 59 different shops. I made the decision to separate the data wrt each shop and create a separate forecasting model for each. The motivation for doing so was both wrt aiding the prediction model to better map the sales function wrt each shop, and the fact that the underlying patterns that drive sales at different shops are most likely unique (if the region in which a shop is located is different from the next shop, we cannot bundle the prediction task, thereby assuming that shops are essentially homogenous). A motivation to bundle the shops into a single prediction model would be if the sales at the shops are not independ (as I have assumed). This would be the case if, for example, the shops are all serviced by a central depo that runs out of stock. Resulting in an interdepence between sales across shops.
</br>
</br>
* __An assumption to note__
    * Explanation: It was assumed that all the products at a shop were sold at least once. In this way, products that do now show up wrt a shop in the received data are assumed not to be stocked there and thus do not require predicting. 

    
__Instructions for executing__: Just run cells from top to bottom. The last cell contains a main function that facilitates the execution of all others

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Prepare data

In [2]:
#Import data
def create_main_df(sales_path: str, items_path: str) -> pd.DataFrame:
    """
    Create the main dataframe containing the raw sales data.
    :param sales_path: String pointing to sales file
    :param items_path: Sting pointing to items file
    :return: pd.DataFrame containg sales data merged with item category data
    """
    #Load csv with sales data
    df_sales = pd.read_csv(sales_path)
    #Load cvs with product category data
    df_items = pd.read_csv(items_path)
    #Merge
    df = pd.merge(df_sales, df_items, on='item_id')
    
    return df

In [3]:
#Aggregate data
def construct_monthly_sales_df(df_func: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate sales data to monthly.
    :param df_func: pd.DataFrame containing the sales data current on a daily level
    :return: pd.DataFrame containing aggregated sales data
    """
    #Create column on which to group data
    df_func['year_month'] = df_func.apply(lambda row:  str(row['year']) + str(row['month']), axis=1)
    #Construct monthly dataframe
    df_func = df_func.groupby(by=['year_month', 'shop_id', 'item_id', 'item_category_id']).agg({'item_cnt_day': 'sum', 'item_price': 'mean'}).reset_index()
    #Rename columns
    df_func.rename(columns={'item_cnt_day': 'item_count_month'}, inplace=True)
    #Round item price
    df_func['item_price'] = df_func['item_price'].round(1)
    #Recreate year and month columns
    df_func['year'] = df_func['year_month'].apply(lambda x:  int(x[:4]))
    df_func['month'] = df_func['year_month'].apply(lambda x:  int(x[4:]))
    #Assign day equal to 1 as placeholder
    df_func['day'] = 1
    #Generate datetime
    df_func['date'] = pd.to_datetime(df_func[['year', 'month', 'day']])
    
    return df_func

In [4]:
#limit the data set to products that sold in 2020
def twenty20_products(df_func: pd.DataFrame, date_limit: datetime) -> pd.DataFrame:
    """
    Limit data to product that were sold in 2020.
    :param df_func: pd.DataFrame containing sales data
    :param date_limit: datetime object indicating relevant cutoff date
    return: pd.DataFrame containing filtered products
    """
    #Identify products
    prods_sold_2020 = df_func[df_func['date']>= date_limit]['item_id'].unique()
    #Limit dataframe
    df_func = df_func.drop(df_func[~df_func['item_id'].isin(prods_sold_2020)].index)
    
    return df_func

In [5]:
#limit the data set to the past 15 months
def limit_data_by_date(df_func: pd.DataFrame, date_limit: datetime) -> pd.DataFrame:
    """
    Limit data to sales after date_limit.
    :param df_func: pd.DataFrame containing sales data
    :param date_limit: datetime object indicating relevant cutoff date
    return: pd.DataFrame containing filtered products
    """
    df_func = df_func.drop(df_func[df_func['date']< date_limit].index)
    
    return df_func

In [6]:
#Drop unwanted columns
def drop_columns(df_func: pd.DataFrame, 
                 columns_to_drop: list) -> pd.DataFrame:
    """
    Remove the columns we do not want to include in the saved data.
    :param df_func: pd.DataFrame containing sales data
    :param columns_to_drop: List containing columns to remove from df
    :return: pd.DataFrame containg filtered columns
    """
    df_func = df_func.drop(columns_to_drop, axis=1)
    
    return df_func

In [7]:
#Add entries for days on which a product was not sold

import warnings
warnings.filterwarnings('ignore')

def get_item_price(df_func: pd.DataFrame, date: datetime) -> int:
    """
    Determine the price of a product wrt a day on which there were no sales for it.
    #ASSUMPTION MADE: On a day that a product was not sold it is assumed that the 
    price was as on its previous sell date. If this is not available, it is set 
    equal to its next sell price instance
    :param df_func: pd.DataFrame containing sales data
    :param date: datetime object indicating date of interest
    :return: Integer specifying price
    """
    price_list_before = df_func[df_func.index <= date]['item_price'].unique()
    price_list_after = df_func[df_func.index > date]['item_price'].unique()   
    if len(price_list_before) > 0:
        return price_list_before[-1]
    else: 
        return price_list_after[0]
    
def get_item_category(df_func: pd.DataFrame) -> int:
    """
    Retrieve the category to which a particular product belongs.
    :param df_func: pd.DataFrame containg sales data
    :return: Integer specifying product category
    """
    category = df_func['item_category_id'].unique()[0]
    return category

def add_no_sale_entries(df_func: pd.DataFrame, 
                        months: pd.Series,
                       unique_shops: list) -> pd.DataFrame:
    """
    For each product-shop combination, add an entry for days on which there were no sales.
    :param df_func: pd.DataFrame containing sales data
    :param months: pd.Series indicating months of interest
    :param unique_shops: List of all shop ids
    :return: Ammended pd.DataFrame now containing zero entries for no sale days
    """
    #TODO: Optimise code further
    for shop in unique_shops:
        print(shop)
        df_shop = df_func[df_func['shop_id'] == shop]
        for i, prod in enumerate(df_shop['item_id'].unique().tolist()):
            df_prod = df_shop[df_shop['item_id']==prod]
            #Determine months with no sales
            months_sold = df_prod.index.tolist()
            months_not_sold = list(set(months) ^ set(months_sold))
            rows_to_add = []
            indexs = []
            for month in months_not_sold:
                #Create entry
                item_price = get_item_price(df_func, month)
                item_category = get_item_category(df_func)
                sold=0
                new_row = {'shop_id': shop, 'item_id':prod, 'item_category_id':item_category, 'item_count_month':sold, 'item_price':round(item_price, 1), 'year':month.year, 'month':month.month}
                rows_to_add.append(new_row)
                indexs.append(month)
            #Append created entry to df
            df_shop  = df_shop.append(pd.DataFrame(rows_to_add,index=indexs,columns=df_shop.columns))
        #Sort df by date
        df_shop = df_shop.sort_index(ascending=True)
        #Write df to file
        df_shop.to_csv('aug_monthed_data_split_by_shop/shop_' + str(shop))
    print('xxxxxxxxxxxxxxxxxxxxxxxxx')

In [8]:
#run notebook
def main():
    """
    Main executable that governs the execution of rest of notebook. 
    """
    #Create main df
    sales_path = 'historic_sales_data.csv'
    items_path = 'items.csv'
    df = create_main_df(sales_path, items_path)
    
    #Aggregate sales to monthly
    df_monthed = construct_monthly_sales_df(df.copy())

    #Limit data to 2020 products
    date_limit = datetime(2020, 1, 1)
    df_monthed = twenty20_products(df_monthed, date_limit)

    #Limit data by date
    date_limit = datetime(2019, 3, 1)
    df_monthed = limit_data_by_date(df_monthed, date_limit)
    
    #Drop unwanted columns
    columns_to_drop = ['year_month', 'day']
    df_monthed = drop_columns(df_monthed, columns_to_drop)
    
    #Add no sale entries
    date_start = datetime(2019, 3, 1)
    date_end = datetime(2020, 5, 1)
    months = pd.date_range(date_start, date_end, freq='MS')
    #Set date as index
    df_monthed = df_monthed.set_index('date')
    #Extract shop list
    unique_shops = df_monthed['shop_id'].unique()
    df_monthed = add_no_sale_entries(df_monthed, months, unique_shops)
    
    print('-------------------------------------------')
    print('-------------------------------------------')

main()

4


KeyboardInterrupt: 